# 🧪 Day 7 - Customer Segmentation with K-Means Clustering

Today’s project focuses on **unsupervised learning** using **K-Means Clustering** to group customers based on their annual income and spending score.

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler